In [14]:
# Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets 
import pandas as pd
import time
import requests
from census import Census
#pd.set_option('display.max_rows', 30000)
# Census API Key
#from config import census_key
#c = Census(census_key, year=2015)
from sqlalchemy import create_engine

## Retrieve Crime Data from FBI Crime Database - UCR

In [5]:
#Store file path for crime data in a variable
crime_data = "Crime Source Data/Offenses_Known_to_Law_Enforcement_by_State_by_City_2015.xls"

#Read data into dataframe
crime_data_df=pd.read_excel(crime_data)

#Extract the state into a list so all rows in "state" column can have a state value
state = crime_data_df["State"]
states=[]
for s in state:
    if pd.isnull(s):
        states.append(current_state)
    else:
        current_state = s
        states.append(current_state)
        
#Add newly created list into the dataframe and remove the old column
crime_data_df["NEW_STATE"] = states
clean_crime_data_df=crime_data_df[["NEW_STATE", "City", "Population", "Violent Crime"]] 
clean_crime_data_df=clean_crime_data_df.rename(columns={"NEW_STATE":"State"})

#Remove any trailing numbers at the end of state and city names
clean_crime_data_df["State"] = clean_crime_data_df["State"].map(lambda x: x.rstrip("1234567890"))
clean_crime_data_df["City"] = clean_crime_data_df["City"].map(lambda x: x.rstrip("1234567890"))

#Calculate the rate of violent crime per capita and remove any city where the violent crime is 0
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]
clean_crime_data_df["Violent Crime Rate"] = (clean_crime_data_df["Violent Crime"]/clean_crime_data_df["Population"])*100

clean_crime_data_df=clean_crime_data_df.sort_values("State")
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]

## Sort Out the Highest and Lowest Crime Rate for Each State

In [12]:
#Sort out the highest crime rate for each state
#variable to hold highest violent crime for each state
violent_crime=[]

#variable to track highest violent crime for each state
violent_crime_track=0

#variable to hold starting state for the loop
current_state='ALABAMA'
MD= 'MARYLAND'
VA='VIRGINIA'
#variable to append city for highest crime rate
city=[]

#variable to append city for highest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if row['State'] == MD or row['State'] == VA:
        crime_rate = row["Violent Crime Rate"] 
        cur_city = row["City"]
        cur_state = row["State"]
        city.append(cur_city)
        state.append(cur_state)
        violent_crime.append(round(crime_rate,2))       
    
#append the data for the last state
#violent_crime.append(highest)
#city.append(cur_city)
#state.append(cur_state)

#Place newly created lists into dictionary and convert to DataFrame
highest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
highest_new_df = pd.DataFrame(highest_new_dict)
lat_lon=pd.read_csv("long_lat_file.csv")
highest_new_df=pd.merge(left=highest_new_df, right=lat_lon, on=['State','City'])
highest_new_df.to_csv("dmv_crime.csv", index=False)
highest_new_df

,State,City,Violent Crime Rate,Latitude,Longitude
0,MARYLAND,La Plata,0.39,38.513875,-77.010340
1,MARYLAND,Mount Rainier,0.52,38.942660,-76.967057
2,MARYLAND,Laurel,0.45,39.093136,-76.881945
3,MARYLAND,Westminster,0.32,39.564201,-76.980701
4,MARYLAND,Manchester,0.02,39.661210,-76.884980
...,...,...,...,...,...
174,VIRGINIA,Falls Church,0.14,38.884177,-77.171752
175,VIRGINIA,Farmville,0.17,37.280694,-78.421284
176,VIRGINIA,Franklin,0.39,36.682635,-76.942233
177,VIRGINIA,Fredericksburg,0.41,38.295135,-77.483689


In [15]:
rds_connection_string = "postgres:<password_here>@localhost:5432/md_va_crime_demo_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
highest_new_df.to_sql(name='crime_rate', con=engine, if_exists='replace', index=False)